In [1]:
import numpy as np
from gurobipy import *

In [2]:
def readFile(nom_file):
    nom_file+=".txt"
    with open(nom_file, 'r') as f:
        l = [[int(num) for num in line.split()] for line in f]
        
    return np.asarray(l)
    f.close()

In [3]:
def pmr(matrice):
    nb_alt,nb_citere=matrice.shape
    matrice_pmr=np.zeros((nb_alt,nb_alt))
    for i in range(nb_alt):
        for j in range(nb_alt):
            if i!=j:               
                cpt=np.ndarray.max(np.array(matrice[i,:]-matrice[j,:]))
                matrice_pmr[i,j]=cpt
    return matrice_pmr


def max_regret(pmrM,l):
    nb_alt=pmrM.shape[0]
    max_regret=np.zeros(nb_alt)
    worst_ad=np.zeros(nb_alt)
    for i in range(nb_alt):
        max_regret[i]=np.ndarray.max(pmrM[i,:])
        worst_ad[i]=int(np.argmax(pmrM[i,:]))
    return max_regret,worst_ad

In [4]:
def Query(poid,sol1,sol2):
    #print(poid*sol1,poid*sol2)
    print(np.sum(poid*sol1),np.sum(poid*sol2))
    if(np.sum(poid*sol1)>np.sum(poid*sol2)):  
        return False
    else:
        return True
    
def expression(q,ligne):
    expr=LinExpr()
    for i in range(len(q)):
        expr+=q[i]*ligne[i]
    return expr

def newPMR(m,matrice,pmr,delete,q):
    matrice=np.delete(matrice,delete,0)
    pmr=np.delete(pmr,delete,0)
    pmr=np.delete(pmr,delete,1)
    #print(pmr)
    a,b=pmr.shape
    for i in range(a):
        for j in range(b):
            if(i!=j):
                expr1=expression(q,matrice[i,:])
                expr2=expression(q,matrice[j,:])
                m.setObjective(expr1-expr2,GRB.MAXIMIZE)
                m.optimize()
                pmr[i,j]=m.objVal
    return matrice,pmr

In [5]:
class MiniMaxRegret(object):
    def __init__(self,matrice,list_poids):
        """
        matrice:matrice de alternatives et leurs evaluations par criteres
        resultat:enregistrer les labels des alternatives
        """
        self.matrice=matrice
        self.nb_alt,self.nb_c=matrice.shape
        self.label=list(range(1,self.nb_alt+1))
        self.poids=list_poids
        self.nb_question=0
        
    
    def MMR(self):
        m = Model("s0")
        #n=matrice.shape[1]
        n=self.nb_c
        var=[]
        for j in range(n):
            var.append(m.addVar(vtype=GRB.CONTINUOUS, name="w%d"%(j+1)))
        expr=LinExpr()
        
        #la premiere constrainte : la somme est egale a 1
        for j in range(n):
            expr+=var[j]
        m.addConstr(expr==1)
        
        i=0
        pmr_matrice=pmr(self.matrice)
        #print(pmr_matrice)
        matrice_mr,worst_ad=max_regret(pmr_matrice,self.label)
        #while(pmr_matrice.shape[0]>=2):
        while(min(matrice_mr)>0):
        
            matrice_mr,worst_ad=max_regret(pmr_matrice,self.label)
            print("matrice_mr is",matrice_mr)
            print("minimum in matrice_mr is",min(matrice_mr))
            sol=np.argmin(matrice_mr)
            print(self.label)
            print("iteration",i+1)
            print("the solution de min-max regret est solution",self.label[sol])
            pire_ad=int(worst_ad[sol])
            print("son pire adversaire est la solution",self.label[pire_ad])
            if(self.label[sol]==self.label[pire_ad]):
                print("best solution est solution",self.label[sol])
                return self.label[sol],self.nb_question
                break
            expr1=expression(var,self.matrice[sol,:])
            expr2=expression(var,self.matrice[pire_ad,:])
            self.nb_question+=1
            if(Query(self.poids,self.matrice[sol,:],self.matrice[pire_ad,:])):
                print(self.matrice[sol,:],self.matrice[pire_ad,:])
                print("solution",self.label[sol],"est plus prefere que",self.label[pire_ad])
                m.addConstr(expr1<=expr2)
                self.label.pop(pire_ad)
                delete=pire_ad
                
            else:
                print("solution",self.label[pire_ad],"est plus prefere que",self.label[sol])
                self.label.pop(sol)
                m.addConstr(expr2<=expr1)
                delete=sol
           
            self.matrice,pmr_matrice=newPMR(m,self.matrice,pmr_matrice,delete,var)
            i+=1
            #print(self.matrice)
            #print(pmr_matrice)
            #print("le nb de questions posees",self.nb_question)
            #return
            

In [6]:
matrice=readFile("exempleinstance2")
poids=np.array([0.326851,0.318176,0.354973])
#poids=np.array([0.6,0.4])
#aleatoire

In [7]:
print(matrice)
print(matrice.shape)

[[32 42 26]
 [26 45 35]
 [39 43 18]
 [45 23 36]
 [37 27 37]]
(5, 3)


In [8]:
m=MiniMaxRegret(matrice,poids)

In [9]:
m.MMR()

Using license file /Users/wty123/gurobi.lic
Academic license - for non-commercial use only
matrice_mr is [19. 22. 20. 19. 19.]
minimum in matrice_mr is 19.0
[1, 2, 3, 4, 5]
iteration 1
the solution de min-max regret est solution 1
son pire adversaire est la solution 4
33.051922000000005 34.805371
[32 42 26] [45 23 36]
solution 1 est plus prefere que 4
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 2 rows, 3 columns and 6 nonzeros
Model fingerprint: 0xf2456d39
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [3e+00, 9e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 2 rows and 3 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds
Optimal objective  6.000000000e+00
Gurobi Optimizer version 9.0.0 build v9.0.0r

  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5775862e+31   3.094828e+30   1.577586e+01      0s
       2   -1.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.03 seconds
Optimal objective -1.000000000e+00
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 3 rows, 3 columns and 9 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.6000000e+30   1.462500e+30   5.600000e+00      0s
       1    6.3636364e-01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.03 seconds
Optimal objective  6.363636364e-01
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 3 rows, 3 columns and 9 nonzeros
Coefficient stat

(3, 3)

In [10]:
matrice=readFile("exempleinstance")
#poids=np.array([0.326851,0.318176,0.354973])
poids=np.array([0.6,0.4])
result=[]
for i in range(matrice.shape[0]):
    result.append(np.sum(matrice[i,:]*poids))

print(result)
print(np.argmin(result)+1)

[428.2, 476.6, 425.4, 607.8, 450.6, 435.8, 794.2, 570.6, 439.8, 425.0]
10


In [11]:
def calcul_opt(matrice,poids):
    result=[]
    for i in range(matrice.shape[0]):
        result.append(np.sum(matrice[i,:]*poids))
    return(np.argmin(result)+1)

In [12]:
print(calcul_opt(matrice,poids))

10


In [13]:
import random
def poid_aleatoire(nb):
    plist=[]
    plist=[random.randint(0,100) for _ in range(nb)]
    total=sum(plist)
    plist=[p/total for p in plist]
    return plist

In [14]:
print(poid_aleatoire(3))

[0.4892086330935252, 0.4316546762589928, 0.07913669064748201]


In [15]:
def test(doc):
    res=[]
    for i in range(2,9):
        nom_file=doc+str(i)
        matrice=readFile(nom_file)
        nbSol,nbCrit=matrice.shape
        poids=poid_aleatoire(nbCrit)
        m=MiniMaxRegret(matrice,poids)
        sol_opt,nb_qu=m.MMR()
        if(sol_opt==calcul_opt(matrice,poids)):
            opt="OUI"
        else:
            opt="NON"
        res.append((nbCrit,nbSol,nb_qu,opt,sol_opt,poids))
    return res

In [16]:
#nombre de question  就是在添加限制条件后 被domine的solution 

In [17]:
res=test("Donnees/tabNDS_10_")

matrice_mr is [ 288.  446.  258.  730.  489.  324. 1561.  658.  338.  269.]
minimum in matrice_mr is 258.0
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
iteration 1
the solution de min-max regret est solution 3
son pire adversaire est la solution 7
410.8913043478261 459.4782608695652
[397 468] [ 139 1777]
solution 3 est plus prefere que 7
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0xa56c2f8c
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+02, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 2 rows and 2 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.1600000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds
Optimal objective  1.160000000e+02
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [2e+02, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3891640e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  1.389164008e+02
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 2 rows, 2 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [3e+01, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3886407e+30   6.534780e+29   1.388641e+00      0s
       1    3.8000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds
Optimal objective  3.800000000e+01
Gurobi Optimizer version 9.0.0 build v9.0.0

Optimal objective  3.600216975e+02
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 2 rows, 2 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+02, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.5973197e+30   6.534780e+29   8.597320e+00      0s
       1    2.7500000e+02   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds
Optimal objective  2.750000000e+02
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 2 rows, 2 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [3e+02, 4e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.9100000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0

Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [2e+02, 4e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.7631653e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds
Optimal objective  2.763165284e+02
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 2 rows, 2 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [8e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6342948e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds
Optimal objective  1.634294831e+02
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 2 rows, 2 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1

  RHS range        [1e+00, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9808551e+30   6.534780e+29   1.980855e+00      0s
       1    5.5000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds
Optimal objective  5.500000000e+01
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 2 rows, 2 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [2e+02, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7100000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds
Optimal objective  1.710000000e+02
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 2 rows, 2 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+01, 2e+01]
  Bounds range     [0e

...

  RHS range        [1e+00, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.8461176e+31   8.165370e+30   7.846118e+01      0s
       4    1.0128324e+03   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.02 seconds
Optimal objective  1.012832442e+03
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 5 rows, 8 columns and 40 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [2e+01, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.8745915e+30   6.883541e+29   5.874592e+00      0s
       1    1.0448265e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds
Optimal objective  1.044826498e+03
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 5 rows, 8 columns and 40 nonzeros
Coefficient statistics:
  Matrix range     [1e+0


Solved in 2 iterations and 0.02 seconds
Optimal objective  8.279067358e+02
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 5 rows, 8 columns and 40 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [8e+01, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.9822280e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  8.982227979e+02
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 5 rows, 8 columns and 40 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [3e+01, 9e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2106845e+30   1.233713e+30   1.210685e+00      0s
       2    4.7704240e+02   0.000000e+00   0.00000

       0    1.8480346e+31   2.020082e+30   1.848035e+01      0s
       5    2.2530747e+02   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds
Optimal objective  2.253074714e+02
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 6 rows, 8 columns and 48 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [3e+01, 9e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.9094029e+31   6.284700e+30   2.909403e+01      0s
       4    3.7160294e+02   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.02 seconds
Optimal objective  3.716029442e+02
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 6 rows, 8 columns and 48 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+02, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [

Optimize a model with 7 rows, 8 columns and 56 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+02, 9e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.0800000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  8.080000000e+02
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 7 rows, 8 columns and 56 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+02, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.2375000e+31   1.098193e+31   2.237500e+01      0s
       3    3.5118575e+02   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds
Optimal objective  3.511857455e+02
Gurobi Optimizer version 9.0.0 build v9.0

       2    6.7100000e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds
Optimal objective  6.710000000e+02
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 8 rows, 8 columns and 64 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [8e+01, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6109375e+31   2.000293e+31   1.610938e+01      0s
       2    8.8568646e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds
Optimal objective  8.856864564e+02
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 8 rows, 8 columns and 64 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [3e+01, 9e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Iteration    Objective       Primal Inf.    Dual I

In [18]:
print(res)

[(2, 10, 4, 'OUI', 5, [0.8043478260869565, 0.1956521739130435]), (3, 10, 5, 'OUI', 2, [0.05, 0.30833333333333335, 0.6416666666666667]), (4, 10, 9, 'OUI', 4, [0.18627450980392157, 0.27941176470588236, 0.28431372549019607, 0.25]), (5, 10, 6, 'OUI', 5, [0.15057915057915058, 0.3281853281853282, 0.3166023166023166, 0.03088803088803089, 0.17374517374517376]), (6, 10, 9, 'OUI', 5, [0.16727272727272727, 0.32, 0.1781818181818182, 0.05818181818181818, 0.14909090909090908, 0.12727272727272726]), (7, 10, 9, 'OUI', 7, [0.07865168539325842, 0.2397003745318352, 0.0898876404494382, 0.13108614232209737, 0.1647940074906367, 0.08239700374531835, 0.21348314606741572]), (8, 10, 9, 'OUI', 4, [0.1794871794871795, 0.16083916083916083, 0.07226107226107226, 0.19347319347319347, 0.05827505827505827, 0.08624708624708624, 0.1351981351981352, 0.11421911421911422])]
